In [1]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2018-10-26 14:00:08--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.204.22.7, 52.1.65.199, 52.1.117.85, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.204.22.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]   5.11M  3.48MB/s    in 1.5s    

2018-10-26 14:00:11 (3.48 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [11]:
get_ipython().system_raw('tensorboard --logdir ./logs/logdir_lenet --host 0.0.0.0 --port 6006 &')
get_ipython().system_raw('./ngrok http 6006 &') # run nrok to tunnel tensorboard to outside world
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://1903403e.ngrok.io


In [3]:
import os
import keras
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.datasets import mnist
from keras.optimizers import Adam
from keras.callbacks import TensorBoard

Using TensorFlow backend.


In [0]:
def lenet(input_shape, num_classes):
    model = Sequential()

    # extract image features by convolution and max pooling layers
    model.add(Conv2D(
        20, kernel_size=5, padding="same",
        input_shape=input_shape, activation="relu"
        ))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(50, kernel_size=5, padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # classify the class by fully-connected layers
    model.add(Flatten())
    model.add(Dense(500, activation="relu"))
    model.add(Dense(num_classes))
    model.add(Activation("softmax"))
    return model

In [0]:
class MNISTDataset():

    def __init__(self):
        self.image_shape = (28, 28, 1)  # image is 28x28x1 (grayscale)
        self.num_classes = 10

    def get_batch(self):
        (x_train, y_train), (x_test, y_test) = mnist.load_data()

        x_train, x_test = [self.preprocess(d) for d in [x_train, x_test]]
        y_train, y_test = [self.preprocess(d, label_data=True) for d in
                           [y_train, y_test]]

        return x_train, y_train, x_test, y_test

    def preprocess(self, data, label_data=False):
        if label_data:
            # convert class vectors to binary class matrices
            data = keras.utils.to_categorical(data, self.num_classes)
        else:
            data = data.astype("float32")
            data /= 255  # convert the value to 0~1 scale
            shape = (data.shape[0],) + self.image_shape  # add dataset length
            data = data.reshape(shape)

        return data

In [0]:
class Trainer():

    def __init__(self, model, loss, optimizer):
        self._target = model
        self._target.compile(
            loss=loss, optimizer=optimizer, metrics=["accuracy"]
            )
        self.verbose = 1
        self.log_dir = "./logs/logdir_lenet"


    def train(self, x_train, y_train, batch_size, epochs, validation_split):
        if os.path.exists(self.log_dir):
            import shutil
            shutil.rmtree(self.log_dir)  # remove previous execution


        self._target.fit(
            x_train, y_train,
            batch_size=batch_size, epochs=epochs,
            validation_split=validation_split,
            callbacks=[TensorBoard(log_dir=self.log_dir)],
            verbose=self.verbose
        )

In [7]:
dataset = MNISTDataset()

x_train, y_train, x_test, y_test = dataset.get_batch()

11493376/11490434 [==============================] - 2s 0us/step


In [8]:
# make model
model = lenet(dataset.image_shape, dataset.num_classes)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 20)        520       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 20)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 50)        25050     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 50)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2450)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               1225500   
_________________________________________________________________
dense_2 (Dense)              (None, 10)                5010      
__________

In [9]:
# train the model

trainer = Trainer(model, loss="categorical_crossentropy", optimizer=Adam())
trainer.train(
    x_train, y_train, batch_size=128, epochs=12, validation_split=0.2
    )


Train on 48000 samples, validate on 12000 samples
Epoch 1/12
48000/48000 [==============================] - 8s 170us/step - loss: 0.1854 - acc: 0.9448 - val_loss: 0.0611 - val_acc: 0.9808
Epoch 2/12
48000/48000 [==============================] - 5s 112us/step - loss: 0.0470 - acc: 0.9853 - val_loss: 0.0481 - val_acc: 0.9849
Epoch 3/12
48000/48000 [==============================] - 5s 112us/step - loss: 0.0328 - acc: 0.9897 - val_loss: 0.0483 - val_acc: 0.9850
Epoch 4/12
48000/48000 [==============================] - 5s 113us/step - loss: 0.0228 - acc: 0.9929 - val_loss: 0.0340 - val_acc: 0.9898
Epoch 5/12
48000/48000 [==============================] - 5s 112us/step - loss: 0.0166 - acc: 0.9946 - val_loss: 0.0321 - val_acc: 0.9896
Epoch 6/12
48000/48000 [==============================] - 5s 112us/step - loss: 0.0145 - acc: 0.9953 - val_loss: 0.0355 - val_acc: 0.9898
Epoch 7/12
48000/48000 [==============================] - 5s 113us/step - loss: 0.0102 - acc: 0.9965 - val_loss: 0.0404 - 

In [10]:
# show result
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.029299741071889047
Test accuracy: 0.9922
